In [ ]:
from dabry.problem import NavigationProblem
from dabry.solver_ef import SolverEFResampling, SolverEF
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.cm as mpl_cm
import numpy as np

plt.rc('font', size=18)
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('legend', fontsize=10)
plt.rc('mathtext', fontset='cm')
plt.rc('text', usetex=True)

#norm = matplotlib.colors.Normalize(vmin=-0.1, vmax=1.3, clip=True)

In [ ]:
def my_figure(solver, ff_on=False, c_levels=np.arange(0, 1.01, 0.1), no_quiver=False, ff_sub=1):
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.axis('equal')
    ax.set_xlim(solver.pb.bl[0], solver.pb.tr[0])
    ax.set_ylim(solver.pb.bl[1], solver.pb.tr[1])
    for traj in solver.trajs:
        ax.plot(*traj.states.T, color='grey', zorder=4)
    for site in solver.solution_sites:
        ax.plot(*site.traj.states.T, color='black', zorder=8)
    if solver.solution_site is not None:
        ax.plot(*solver.solution_site.traj.states.T, color='red', zorder=8)
    if not ff_on:
        c = ax.contourf(solver._cost_map.grid_vectors[1:-1, 1:-1, 0], solver._cost_map.grid_vectors[1:-1, 1:-1, 1],
                    solver._cost_map.values[1:-1, 1:-1], levels=c_levels, cmap='jet', zorder=4, alpha=0.5)
    else:
        ff = solver.pb.model.ff
        grid_vectors_ff = np.stack(
            np.meshgrid(
                np.linspace(
                    ff.bounds[-2, 0], 
                    ff.bounds[-2, 1], 
                    ff.values.shape[-3]
                ),
                np.linspace(
                    ff.bounds[-1, 0], 
                    ff.bounds[-1, 1], 
                    ff.values.shape[-2]
                ), indexing= 'ij'), -1)
        grid_vectors_ff = grid_vectors_ff[::ff_sub, ::ff_sub]
        ff_values = ff.values[::ff_sub, ::ff_sub]
        ff_norms = np.linalg.norm(ff_values, axis=-1)
        ax.pcolormesh(grid_vectors_ff[..., 0], grid_vectors_ff[..., 1], 
                      np.where(ff_norms > 0.05, ff_norms, np.nan), zorder=2,
                      shading='gouraud', cmap='turbo')
        rect = patches.Rectangle(solver.pb.bl, solver.pb.tr[0] - solver.pb.bl[0], solver.pb.tr[1] - solver.pb.bl[1],
                                alpha=0.3, color='white', zorder=3)
        ax.add_patch(rect)
        if not no_quiver:
            ax.quiver(grid_vectors_ff[..., 0], grid_vectors_ff[..., 1], ff_values[..., 0], ff_values[..., 1], zorder=4)
        
    c = ax.contour(solver._cost_map.grid_vectors[1:-1, 1:-1, 0], solver._cost_map.grid_vectors[1:-1, 1:-1, 1],
                solver._cost_map.values[1:-1, 1:-1], levels=c_levels, 
                   alpha=1, colors=((0.2, 0.2, 0.2),) if not ff_on else 'black', zorder=5)
    ax.clabel(c, c.levels, inline=True, fontsize=15)
    obs = solver.pb.obstacles[0]
    grid_vectors_obs = np.stack(
        np.meshgrid(
            np.linspace(
                obs.bounds[-2, 0], 
                obs.bounds[-2, 1], 
                obs.values.shape[-2]
            ),
            np.linspace(
                obs.bounds[-1, 0], 
                obs.bounds[-1, 1], 
                obs.values.shape[-1]
            ), indexing= 'ij'), -1)
    ax.contourf(grid_vectors_obs[..., 0], grid_vectors_obs[..., 1], obs.values, levels=[-100, 0],
                colors='purple', extend='min', alpha=0.5, zorder=4)
    ax.contour(grid_vectors_obs[..., 0], grid_vectors_obs[..., 1], obs.values, levels=0, colors='purple', zorder=4)
    ax.scatter(*solver.pb.x_init, color='black', edgecolor='white', s=100, zorder=10)
    ax.scatter(*solver.pb.x_target, color='black', edgecolor='white', s=200, marker='*', zorder=10)
    circ = patches.Circle(solver.pb.x_target, solver.target_radius, facecolor='none', edgecolor='black', linewidth=1, zorder=8)
    ax.add_patch(circ)
    return fig, ax

# prop_cycle = plt.rcParams['axes.prop_cycle']
# colors = prop_cycle.by_key()['color']

def my_figure_details(solver, ff_on=False, c_levels=np.arange(0, 1.01, 0.1), no_quiver=False, ff_sub=1):
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.axis('equal')
    ax.set_xlim(solver.pb.bl[0], solver.pb.tr[0])
    ax.set_ylim(solver.pb.bl[1], solver.pb.tr[1])
    labelled = []
    for site in solver.sites.values():
        kwargs = {}
        if not site.depth in labelled:
            labelled.append(site.depth)
            kwargs['label'] = f'Depth {site.depth:0>2}'
        ax.plot(*site.traj.states.T, color=matplotlib.colormaps.get_cmap('nipy_spectral_r')(site.depth/11), zorder=4, **kwargs)
    # for site in solver.solution_sites:
    #     ax.plot(*site.traj.states.T, color='black', zorder=8)
    # if solver.solution_site is not None:
    #     ax.plot(*solver.solution_site.traj.states.T, color='red', zorder=8)
    if not ff_on:
        pass
    else:
        ff = solver.pb.model.ff
        grid_vectors_ff = np.stack(
            np.meshgrid(
                np.linspace(
                    ff.bounds[-2, 0], 
                    ff.bounds[-2, 1], 
                    ff.values.shape[-3]
                ),
                np.linspace(
                    ff.bounds[-1, 0], 
                    ff.bounds[-1, 1], 
                    ff.values.shape[-2]
                ), indexing= 'ij'), -1)
        grid_vectors_ff = grid_vectors_ff[::ff_sub, ::ff_sub]
        ff_values = ff.values[::ff_sub, ::ff_sub]
        ff_norms = np.linalg.norm(ff_values, axis=-1)
        ax.pcolormesh(grid_vectors_ff[..., 0], grid_vectors_ff[..., 1], 
                      np.where(ff_norms > 0.05, ff_norms, np.nan), zorder=2,
                      shading='gouraud', cmap='turbo')
        rect = patches.Rectangle(solver.pb.bl, solver.pb.tr[0] - solver.pb.bl[0], solver.pb.tr[1] - solver.pb.bl[1],
                                alpha=0.3, color='white', zorder=3)
        ax.add_patch(rect)
        if not no_quiver:
            ax.quiver(grid_vectors_ff[..., 0], grid_vectors_ff[..., 1], ff_values[..., 0], ff_values[..., 1], zorder=4)
        
    # c = ax.contour(solver._cost_map.grid_vectors[1:-1, 1:-1, 0], solver._cost_map.grid_vectors[1:-1, 1:-1, 1],
    #             solver._cost_map.values[1:-1, 1:-1], levels=c_levels, 
    #                alpha=1, colors=((0.2, 0.2, 0.2),) if not ff_on else 'black', zorder=5)
    # ax.clabel(c, c.levels, inline=True, fontsize=15)
    obs = solver.pb.obstacles[0]
    grid_vectors_obs = np.stack(
        np.meshgrid(
            np.linspace(
                obs.bounds[-2, 0], 
                obs.bounds[-2, 1], 
                obs.values.shape[-2]
            ),
            np.linspace(
                obs.bounds[-1, 0], 
                obs.bounds[-1, 1], 
                obs.values.shape[-1]
            ), indexing= 'ij'), -1)
    ax.contourf(grid_vectors_obs[..., 0], grid_vectors_obs[..., 1], obs.values, levels=[-100, 0],
                colors='purple', extend='min', alpha=0.5, zorder=2)
    ax.contour(grid_vectors_obs[..., 0], grid_vectors_obs[..., 1], obs.values, levels=0, colors='purple', zorder=2)
    ax.scatter(*solver.pb.x_init, color='black', edgecolor='white', s=100, zorder=10)
    ax.scatter(*solver.pb.x_target, color='black', edgecolor='white', s=200, marker='*', zorder=10)
    circ = patches.Circle(solver.pb.x_target, solver.target_radius, facecolor='none', edgecolor='black', linewidth=1, zorder=8)
    ax.add_patch(circ)
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    return fig, ax

# Custom problems

## Disk

In [ ]:
from dabry.flowfield import ZeroFF
from dabry.obstacle import CircleObs, Union, discretize_obs, DiscreteObs
v_a = 1

x_init = np.array((0.15, 0.5))
x_target = np.array((0.85, 0.5))
bl = np.array((0, 0))
tr = np.array((1, 1))

ff = ZeroFF()

obs = CircleObs(np.array((0.5, 0.5)), 0.19)
obs_d = discretize_obs(obs, (101, 101), bl=bl, tr=tr)

pb = NavigationProblem(ff, x_init, x_target, v_a, bl=bl, tr=tr, name='Circle obstacle', obstacles=[obs_d])

In [ ]:
solver_circle = SolverEFResampling(pb, free_max_step=False)
solver_circle.solve()

In [ ]:
fig, ax = my_figure(solver_circle)
#fig.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_simple/disk.pdf')
plt.show()

In [ ]:
fig, ax = my_figure_details(solver_circle)
# fig.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_simple/disk_details.pdf', 
#             bbox_inches='tight')
plt.show()

## Concave

In [ ]:
import shapely

In [ ]:
theta = np.linspace(np.pi/2, 3*np.pi/2, 100)
center = np.array((0.5, 0.5))
radius = 0.25
dist = 0.025
points = center + radius * np.array((np.cos(theta), np.sin(theta))).T
arc = shapely.LineString(points)

shape = (101, 101)
grid_vectors = np.stack(np.meshgrid(np.linspace(bl[0], tr[0], shape[0]),
                                    np.linspace(bl[1], tr[1], shape[1]), indexing='ij'), -1)
values = np.zeros(shape)
for i in range(shape[0]):
    for j in range(shape[1]):
        values[i, j] = arc.distance(shapely.Point(grid_vectors[i, j])) - dist

In [ ]:
plt.contour(grid_vectors[..., 0], grid_vectors[..., 1], values, levels=[0])

In [ ]:
from dabry.flowfield import ZeroFF
from dabry.obstacle import CircleObs, Union, discretize_obs, DiscreteObs
v_a = 1

x_init = np.array((0.15, 0.5))
x_target = np.array((0.85, 0.5))
bl = np.array((0, 0))
tr = np.array((1, 1))

ff = ZeroFF()

obs_d = DiscreteObs(values, np.array((bl, tr)).T)

pb = NavigationProblem(ff, x_init, x_target, v_a, bl=bl, tr=tr, name='Concave', obstacles=[obs_d])

In [ ]:
solver_concave = SolverEFResampling(pb, free_max_step=False, total_duration=1.11)
solver_concave.solve()

In [ ]:
fig, ax = my_figure(solver_concave)
#plt.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_simple/concave.pdf')
plt.show()

## Bars

In [ ]:
dist = 0.025
points = np.array((np.linspace(0.3, 0.3, 50), np.linspace(0.4, 1.1, 50))).T
line1 = shapely.LineString(points)

shape = (101, 101)
grid_vectors = np.stack(np.meshgrid(np.linspace(bl[0], tr[0], shape[0]),
                                    np.linspace(bl[1], tr[1], shape[1]), indexing='ij'), -1)
values1 = np.zeros(shape)
for i in range(shape[0]):
    for j in range(shape[1]):
        values1[i, j] = line1.distance(shapely.Point(grid_vectors[i, j])) - dist

points = np.array((np.linspace(0.6, 0.6, 50), np.linspace(-0.1, 0.6, 50))).T
line2 = shapely.LineString(points)

values2 = np.zeros(shape)
for i in range(shape[0]):
    for j in range(shape[1]):
        values2[i, j] = line2.distance(shapely.Point(grid_vectors[i, j])) - dist

values3 = np.zeros(shape)
for i in range(shape[0]):
    for j in range(shape[1]):
        values3[i, j] = 0.49 - np.max(np.abs(grid_vectors[i, j] - np.array((0.5, 0.5))))

values = np.minimum(np.minimum(values1, values2), values3)

In [ ]:
plt.contour(grid_vectors[..., 0], grid_vectors[..., 1], values, levels=[0])

In [ ]:
from dabry.flowfield import DiscreteFF
from dabry.misc import Coords
v_a = 1

x_init = np.array((0.15, 0.5))
x_target = np.array((0.85, 0.5))
bl = np.array((0, 0))
tr = np.array((1, 1))

#ff = DiscreteFF(ff_values, np.array((bl, tr)).T, Coords.CARTESIAN)
ff = ZeroFF()

obs_d = DiscreteObs(values, np.array((bl, tr)).T)

pb = NavigationProblem(ff, x_init, x_target, v_a, bl=bl, tr=tr, name='Bars flow', obstacles=[obs_d])

In [ ]:
solver_bars = SolverEFResampling(pb, free_max_step=False, total_duration=1.2, tangency_tol=1)
solver_bars.solve()

In [ ]:
fig, ax = my_figure(solver_bars)
#plt.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_simple/bars.pdf')
plt.show()

In [ ]:
shape = (51, 51)
grid_vectors = np.stack(np.meshgrid(np.linspace(bl[0], tr[0], shape[0]),
                                    np.linspace(bl[1], tr[1], shape[1]), indexing='ij'), -1)
factor = 20/3

center = np.array((0.6, 0.6))
radius = 0.15
ff_values = np.stack((
    -factor*np.maximum(radius - np.linalg.norm(grid_vectors - center, axis=-1), 0),
    np.zeros(grid_vectors.shape[:-1])), -1)

center = np.array((0.3, 0.3))
radius = 0.15
ff_values = ff_values + np.stack((
    -factor*np.maximum(radius - np.linalg.norm(grid_vectors - center, axis=-1), 0),
    np.zeros(grid_vectors.shape[:-1])), -1)

In [ ]:
np.linalg.norm(ff_values, axis=-1).max()

In [ ]:
ff = DiscreteFF(ff_values, np.array((bl, tr)).T, Coords.CARTESIAN)

In [ ]:
pb = NavigationProblem(ff, x_init, x_target, v_a, bl=bl, tr=tr, name='Bars flow', obstacles=[obs_d])

In [ ]:
solver_bars_flow = SolverEFResampling(pb, free_max_step=False, total_duration=1.2, tangency_tol=np.pi/2)
solver_bars_flow.solve()

In [ ]:
fig, ax = my_figure(solver_bars_flow, c_levels=np.arange(0, 1.21, 0.1), ff_on=True, no_quiver=True)
plt.arrow(0.3+0.075, 0.3, -0.05, 0, zorder=9, width=0.025, edgecolor='black', facecolor='none', alpha=0.7, overhang=0.2)
plt.arrow(0.6+0.075, 0.6, -0.05, 0, zorder=9, width=0.025, edgecolor='black', facecolor='none', alpha=0.7, overhang=0.2)
#plt.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_simple/bars_flow.png', dpi=300)
plt.show()

In [ ]:
from dabry.flowfield import StateLinearFF
srf = 1.

f = 1.
x_init = f * np.array([0., 0.])
x_target = f * np.array([1., 0.])

g = 1.
gradient = np.array([[0., g], [0., 0.]])
origin = np.array([0., 0.])
value_origin = np.array([0., 0.])

bl = f * np.array([-0.2, -0.7])
tr = f * np.array([1.2, 0.7])

ff_a = StateLinearFF(gradient, origin, value_origin)

ff = DiscreteFF.from_ff(ff_a, np.array((bl, tr)).T)

In [ ]:
theta = np.linspace(np.pi/2, 3*np.pi/2, 100)
center = np.array((0.5, 0))
radius = 0.25
dist = 0.025
points = center + np.stack((np.linspace(0, 0, 51), np.linspace(-0.1, 0.2, 51)), -1)
bar = shapely.LineString(points)

shape = (101, 101)
grid_vectors = np.stack(np.meshgrid(np.linspace(bl[0], tr[0], shape[0]),
                                    np.linspace(bl[1], tr[1], shape[1]), indexing='ij'), -1)
values = np.zeros(shape)
for i in range(shape[0]):
    for j in range(shape[1]):
        values[i, j] = bar.distance(shapely.Point(grid_vectors[i, j])) - dist

In [ ]:
plt.contour(grid_vectors[..., 0], grid_vectors[..., 1], values, levels=[0])

In [ ]:
obs = DiscreteObs(values, np.array((bl, tr)).T)

pb = NavigationProblem(ff, x_init, x_target, srf, bl=bl, tr=tr, name='Linear Obstacle', obstacles=[obs])

In [ ]:
solver_linear = SolverEFResampling(pb, free_max_step=False, tangency_tol=np.pi/2)
solver_linear.solve()

In [ ]:
fig, ax = my_figure(solver_linear, c_levels=np.arange(0, 1.21, 0.1), ff_on=True, ff_sub=4)
#plt.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_simple/linear_obs.pdf')
plt.show()